import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization,Input,ReLU,add,MaxPool1D,GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score,mean_absolute_error
from tensorflow.keras import callbacks,Model,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=900
cls_num=2
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=WormsTwoClass0(i)

os.chdir(cwd)
clses_lens=np.array([])
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (1,cls_num):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
btch=int(min(clses_lens))

first_train_shape= (77, 901)
first_test_shape= (181, 901)
classes_quantity= 2
tr_lbls=	 {1, 2}
Count_labels= [33 44]
max(train_feature_Altitude)= 4.1961
min(train_feature_Altitude)= -4.8873
first_train_sample=
 [ 1.        -0.77859   -0.74444   -0.72546   -0.70269   -0.71408
 -0.72167   -0.73305   -0.71787   -0.73685   -0.76341   -0.77479
 -0.76341   -0.74823   -0.72546   -0.73305   -0.73305   -0.74064
 -0.74444   -0.76341   -0.7672    -0.76341   -0.75582   -0.74823
 -0.74823   -0.75582   -0.74823   -0.75962   -0.75203   -0.74823
 -0.74823   -0.73305   -0.73305   -0.74064   -0.771     -0.80895
 -0.83931   -0.86587   -0.87725   -0.87346   -0.919     -0.86207
 -0.86207   -0.85069   -0.82792   -0.85069   -0.78238   -0.7672
 -0.74444   -0.72546   -0.75203   -0.6951    -0.6951    -0.72167
 -0.72926   -0.74444   -0.74444   -0.79756   -0.75203   -0.67233
 -0.64577   -0.67613   -0.65716   -0.61541   -0.63818   -0.63439
 -0.59264   -0.61162   -0.5547    -0.42567   -0.30044   -0.21316
 -0.22455 

In [3]:
np.shape(ecg)

(66, 901)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 33	2 44	
 max = 44


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (33, 901)
Up to class  2 train shape =  (77, 901)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0 33 44]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  4.1375
min magnitude class 1  =  -4.037800000000001
after normalizing >>
max magnitude class 1  =  0.9870973424048264
min magnitude class 1  =  -0.8129555012440277

 cls 2  >> 
max magnitude class 2  =  4.8591000000000015
min magnitude class 2  =  -4.3114
after normalizing >>
max magnitude class 2  =  1.1459805799590468
min magnitude class 2  =  -0.8731972609375345
each class and its windows =
1 76	2 105	
 max instance in classes of test windows = 105


In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.8731972609375345
max tst = 1.1459805799590468


In [14]:
len(wndws_test)

181

In [15]:
np.shape(wndws_test)

(181, 901)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2], dtype=int16)


<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(33, 901)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (77, 900)
y_train =>  (77,)
X_test  =>  (181, 900)
y_test  =>  (181,)
X_valid  =>  (77, 900)
y_valid  =>  (77,)


In [22]:
if min(y_train)==0:
    print(' class was started from 0=Zero ')
else:
    print('!!!!!! class Should start from 0=Zero !!!!!!')
    print("!!!!!!   Reduce 1 From Class Number    !!!!!")
    y_train=np.array(y_train-1)
    y_test=np.array(y_test-1)
    y_valid=np.array(y_valid-1)
print('final labels=',set(y_train))

!!!!!! class Should start from 0=Zero !!!!!!
!!!!!!   Reduce 1 From Class Number    !!!!!
final labels= {0, 1}


In [23]:
y_train_cat=to_categorical(y_train)#[:,1:]
y_valid_cat=to_categorical(y_valid)#[:,1:]
y_test_cat=to_categorical(y_test)#[:,1:]


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

<h1><center><div style="direction:rtl;font-family:B Nazanin">Network</div></center></h1>

In [24]:
# Import ResNet1D Module
#from ResNet_1DCNN import ResNet
#from ResNeXt_1DCNN import ResNeXt as ResNet
from SE_ResNeXt_1DCNN import SEResNeXt as ResNet

"Configurations for ResNet in Regression Mode"
length = X_train.shape[1]   # Number of Features (or length of the signal)
model_width = 64           # Number of Filter or Kernel in the Input Layer
num_channel = 1             # Number of Input Channels
problem_type = 'Classification' # Regression or Classification
output_number = cls_num     # Number of Outputs in the Regression Mode
Regression_Model = ResNet(length, num_channel, model_width, problem_type=problem_type, output_nums=output_number).SEResNeXt50() # Build Model
# SEResNext Models supported: SEResNeXt18, SEResNeXt34, SEResNeXt50, SEResNeXt101, SEResNeXt152, 
Regression_Model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 100, restore_best_weights = True)
history = Regression_Model.fit(X_train, y_train_cat, epochs=500, batch_size=btch, verbose=1, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping]) #,validation_split=0.2,callbacks =[callbacks]

Epoch 1/500
3/3 [==============================] - 41s 2s/step - loss: 1.3792 - accuracy: 0.6364 - val_loss: 5654.6587 - val_accuracy: 0.5714
Epoch 2/500
3/3 [==============================] - 1s 459ms/step - loss: 1.3301 - accuracy: 0.5325 - val_loss: 5240.2290 - val_accuracy: 0.5714
Epoch 3/500
3/3 [==============================] - 1s 447ms/step - loss: 0.6540 - accuracy: 0.6234 - val_loss: 2426.2158 - val_accuracy: 0.5714
Epoch 4/500
3/3 [==============================] - 1s 442ms/step - loss: 0.6004 - accuracy: 0.5844 - val_loss: 1000.7851 - val_accuracy: 0.5714
Epoch 5/500
3/3 [==============================] - 1s 441ms/step - loss: 0.5534 - accuracy: 0.7403 - val_loss: 471.5626 - val_accuracy: 0.5714
Epoch 6/500
3/3 [==============================] - 1s 443ms/step - loss: 0.6065 - accuracy: 0.6364 - val_loss: 183.7463 - val_accuracy: 0.5714
Epoch 7/500
3/3 [==============================] - 1s 508ms/step - loss: 0.4976 - accuracy: 0.7013 - val_loss: 43.7867 - val_accuracy: 0.584

3/3 [==============================] - 1s 501ms/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 2.5388 - val_accuracy: 0.5584
Epoch 59/500
3/3 [==============================] - 1s 486ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 3.0771 - val_accuracy: 0.5584
Epoch 60/500
3/3 [==============================] - 1s 486ms/step - loss: 7.6116e-04 - accuracy: 1.0000 - val_loss: 3.1894 - val_accuracy: 0.5714
Epoch 61/500
3/3 [==============================] - 1s 483ms/step - loss: 3.7566e-04 - accuracy: 1.0000 - val_loss: 3.1873 - val_accuracy: 0.5714
Epoch 62/500
3/3 [==============================] - 1s 475ms/step - loss: 3.4420e-04 - accuracy: 1.0000 - val_loss: 3.1648 - val_accuracy: 0.5714
Epoch 63/500
3/3 [==============================] - 1s 454ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 2.9179 - val_accuracy: 0.5584
Epoch 64/500
3/3 [==============================] - 1s 459ms/step - loss: 0.2291 - accuracy: 0.9740 - val_loss: 5.7633 - val_accuracy: 0.5714
Epoch 6

3/3 [==============================] - 1s 454ms/step - loss: 0.3283 - accuracy: 0.9221 - val_loss: 13.0322 - val_accuracy: 0.6234
Epoch 116/500
3/3 [==============================] - 1s 453ms/step - loss: 0.2223 - accuracy: 0.8831 - val_loss: 5.3983 - val_accuracy: 0.5974
Epoch 117/500
3/3 [==============================] - 1s 521ms/step - loss: 0.1264 - accuracy: 0.9481 - val_loss: 1.0282 - val_accuracy: 0.7013
Epoch 118/500
3/3 [==============================] - 1s 465ms/step - loss: 0.1680 - accuracy: 0.9221 - val_loss: 17.8145 - val_accuracy: 0.5974
Epoch 119/500
3/3 [==============================] - 1s 466ms/step - loss: 0.0416 - accuracy: 1.0000 - val_loss: 14.0415 - val_accuracy: 0.5455
Epoch 120/500
3/3 [==============================] - 1s 458ms/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 10.8025 - val_accuracy: 0.5584
Epoch 121/500
3/3 [==============================] - 1s 457ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 8.1547 - val_accuracy: 0.5584
Epoch 122

3/3 [==============================] - 1s 533ms/step - loss: 2.4551e-04 - accuracy: 1.0000 - val_loss: 0.1016 - val_accuracy: 0.9610
Epoch 173/500
3/3 [==============================] - 1s 457ms/step - loss: 1.4023e-04 - accuracy: 1.0000 - val_loss: 0.1038 - val_accuracy: 0.9610
Epoch 174/500
3/3 [==============================] - 1s 505ms/step - loss: 1.0985e-04 - accuracy: 1.0000 - val_loss: 0.1004 - val_accuracy: 0.9740
Epoch 175/500
3/3 [==============================] - 1s 512ms/step - loss: 6.1009e-04 - accuracy: 1.0000 - val_loss: 0.0342 - val_accuracy: 0.9870
Epoch 176/500
3/3 [==============================] - 1s 459ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 1.0201 - val_accuracy: 0.8052
Epoch 177/500
3/3 [==============================] - 1s 452ms/step - loss: 0.3135 - accuracy: 0.9481 - val_loss: 2.8202 - val_accuracy: 0.7792
Epoch 178/500
3/3 [==============================] - 1s 444ms/step - loss: 0.5910 - accuracy: 0.8831 - val_loss: 6.4345 - val_accuracy: 0.76

3/3 [==============================] - 1s 440ms/step - loss: 8.7749e-04 - accuracy: 1.0000 - val_loss: 0.1825 - val_accuracy: 0.9351
Epoch 230/500
3/3 [==============================] - 1s 438ms/step - loss: 3.0036e-04 - accuracy: 1.0000 - val_loss: 0.1478 - val_accuracy: 0.9351
Epoch 231/500
3/3 [==============================] - 1s 439ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0410 - val_accuracy: 0.9740
Epoch 232/500
3/3 [==============================] - 1s 436ms/step - loss: 3.0373e-04 - accuracy: 1.0000 - val_loss: 0.0330 - val_accuracy: 0.9740
Epoch 233/500
3/3 [==============================] - 1s 446ms/step - loss: 2.0314e-04 - accuracy: 1.0000 - val_loss: 0.0199 - val_accuracy: 0.9870
Epoch 234/500
3/3 [==============================] - 1s 520ms/step - loss: 1.0363e-04 - accuracy: 1.0000 - val_loss: 0.0116 - val_accuracy: 1.0000
Epoch 235/500
3/3 [==============================] - 1s 452ms/step - loss: 1.7744e-04 - accuracy: 1.0000 - val_loss: 0.0063 - val_accura

Epoch 286/500
3/3 [==============================] - 1s 466ms/step - loss: 0.1051 - accuracy: 0.9610 - val_loss: 0.4758 - val_accuracy: 0.8831
Epoch 287/500
3/3 [==============================] - 1s 471ms/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 0.3182 - val_accuracy: 0.8961
Epoch 288/500
3/3 [==============================] - 1s 456ms/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.2571 - val_accuracy: 0.9221
Epoch 289/500
3/3 [==============================] - 1s 447ms/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.1545 - val_accuracy: 0.9351
Epoch 290/500
3/3 [==============================] - 1s 445ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.1326 - val_accuracy: 0.9610
Epoch 291/500
3/3 [==============================] - 1s 453ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.1151 - val_accuracy: 0.9610
Epoch 292/500
3/3 [==============================] - 1s 457ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.1869 - val_accuracy: 0.9481

In [25]:
predict_x=Regression_Model.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                 # function from augment.py to remove 0 index predictions

#y_pred_crs = model_Res.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

6/6 [==============================] - 4s 152ms/step
array([[48, 28],
       [28, 77]], dtype=int64)
              precision    recall  f1-score   support

           0       0.63      0.63      0.63        76
           1       0.73      0.73      0.73       105

    accuracy                           0.69       181
   macro avg       0.68      0.68      0.68       181
weighted avg       0.69      0.69      0.69       181

